In [1]:
!pip install openai
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.9 MB/s eta 0:00:00


In [2]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

## Prepare Dataset
{
"prompt": "my prompt ->",
"completion": "the answer of the prompt. \n"
}


In [29]:
training_data = [
	{
    	"prompt": "What is the capital of France?->",
    	"completion": """ The capital of France is Paris.\n"""
	},
	{
    	"prompt": "What is the primary function of the heart?->",
    	"completion": """ The primary function of the heart is to pump blood throughout the body.\n"""
	},
	{
    	"prompt": "What is photosynthesis?->",
    	"completion": """ Photosynthesis is the process by which green plants and some other organisms convert sunlight into chemical energy stored in the form of glucose.\n"""
	},
	{
    	"prompt": "Who wrote the play 'Romeo and Juliet'?->",
    	"completion": """ William Shakespeare wrote the play 'Romeo and Juliet'.\n"""
	},
	{
    	"prompt": "Which element has the atomic number 1?->",
    	"completion": """ Hydrogen has the atomic number 1.\n"""
	},
	{
    	"prompt": "What is the largest planet in our solar system?->",
    	"completion": """ Jupiter is the largest planet in our solar system.\n"""
	},
	{
    	"prompt": "What is the freezing point of water in Celsius?->",
    	"completion": """ The freezing point of water in Celsius is 0 degrees.\n"""
	},
	{
    	"prompt": "What is the square root of 144?->",
    	"completion": """ The square root of 144 is 12.\n"""
	},
	{
    	"prompt": "Who is the author of 'To Kill a Mockingbird'?->",
    	"completion": """ The author of 'To Kill a Mockingbird' is Harper Lee.\n"""
	},
	{
    	"prompt": "What is the smallest unit of life?->",
    	"completion": """ The smallest unit of life is the cell.\n"""
	},
	{
    	"prompt": "Who is the priminister of Thailand?->",
    	"completion": """ Mr. Pita Limchareonrat\n"""
	}
]


## Prepare files to upload

In [30]:
import json

training_file_name = "training_data.jsonl"

def prepare_data(dictionary_data, final_file_name):
  with open(final_file_name, 'w') as outfile:
    for entry in dictionary_data:
      json.dump(entry, outfile)
      outfile.write('\n')

prepare_data(training_data, "training_data.jsonl")

In [31]:
def upload_data_to_OpenAI(filename):
  with open(filename, "rb") as dataset_file:
    response = openai.Dataset.create(
        file=dataset_file,
        purpose="fine-tuning",
        name="questions_answers_dataset"
    )
  return response["id"]


In [32]:
!openai tools fine_tunes.prepare_data -f "training_data.jsonl"

Analyzing...

- Your file contains 11 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- All prompts end with suffix `?->`
- All prompts start with prefix `Wh`
- All completions end with suffix `\n`

No remediations found.

You can use your file for fine-tuning:
> openai api fine_tunes.create -t "training_data.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string `?->` for the model to start generating completions, rather than continuing with the prompt. Make sure to include `stop=["\n"]` so that the generated texts ends at the expected place.
Once your model starts training, it'll approximately take 2.59 minutes to train a `curie` model, and less for `ada` and `babbage`. Queue will approximately take half an hour per job ahead of you.


In [35]:
!openai api fine_tunes.create -t "training_data.jsonl" -m "curie"

Upload progress: 100% 1.41k/1.41k [00:00<00:00, 1.30Mit/s]
Uploaded file from training_data.jsonl: file-sTzqEgYgUrYKg4Wawz4PswkA
Created fine-tune: ft-NiHENGzjzSIS37Pc8HAYp7Fa
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-06-29 06:03:47] Created fine-tune: ft-NiHENGzjzSIS37Pc8HAYp7Fa

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-NiHENGzjzSIS37Pc8HAYp7Fa



In [22]:
!openai api fine_tunes.follow -i ft-NiHENGzjzSIS37Pc8HAYp7Fa

[2023-06-29 05:33:16] Created fine-tune: ft-nBpBzRcC3QZvXLftT2gUSISs

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-nBpBzRcC3QZvXLftT2gUSISs



In [36]:
# List all created fine-tunes
!openai api fine_tunes.list



{
  "object": "list",
  "data": [
    {
      "object": "fine-tune",
      "id": "ft-nBpBzRcC3QZvXLftT2gUSISs",
      "hyperparams": {
        "n_epochs": 4,
        "batch_size": null,
        "prompt_loss_weight": 0.01,
        "learning_rate_multiplier": null
      },
      "organization_id": "org-5DvmRlg7kGvkcggchTvEJDF0",
      "model": "davinci",
      "training_files": [
        {
          "object": "file",
          "id": "file-kRRpt3KvorsOei3GZyFPDWQy",
          "purpose": "fine-tune",
          "filename": "training_data.jsonl",
          "bytes": 1310,
          "created_at": 1688016796,
          "status": "processed",
          "status_details": null
        }
      ],
      "validation_files": [],
      "result_files": [],
      "created_at": 1688016796,
      "updated_at": 1688018323,
      "status": "cancelled",
      "fine_tuned_model": null
    },
    {
      "object": "fine-tune",
      "id": "ft-NiHENGzjzSIS37Pc8HAYp7Fa",
      "hyperparams": {
        "n_epochs":

In [39]:
# Retrieve the state of a fine-tune. The resulting object includes
# job status (which can be one of pending, running, succeeded, or failed)
# and other information
!openai api fine_tunes.get -i ft-NiHENGzjzSIS37Pc8HAYp7Fa


{
  "object": "fine-tune",
  "id": "ft-NiHENGzjzSIS37Pc8HAYp7Fa",
  "hyperparams": {
    "n_epochs": 4,
    "batch_size": null,
    "prompt_loss_weight": 0.01,
    "learning_rate_multiplier": null
  },
  "organization_id": "org-5DvmRlg7kGvkcggchTvEJDF0",
  "model": "curie",
  "training_files": [
    {
      "object": "file",
      "id": "file-sTzqEgYgUrYKg4Wawz4PswkA",
      "purpose": "fine-tune",
      "filename": "training_data.jsonl",
      "bytes": 1405,
      "created_at": 1688018627,
      "status": "processed",
      "status_details": null
    }
  ],
  "validation_files": [],
  "result_files": [],
  "created_at": 1688018627,
  "updated_at": 1688018627,
  "status": "pending",
  "fine_tuned_model": null,
  "events": [
    {
      "object": "fine-tune-event",
      "level": "info",
      "message": "Created fine-tune: ft-NiHENGzjzSIS37Pc8HAYp7Fa",
      "created_at": 1688018627
    }
  ]
}


In [28]:
# Cancel a job
!openai api fine_tunes.cancel -i ft-nBpBzRcC3QZvXLftT2gUSISs

{
  "object": "fine-tune",
  "id": "ft-nBpBzRcC3QZvXLftT2gUSISs",
  "hyperparams": {
    "n_epochs": 4,
    "batch_size": null,
    "prompt_loss_weight": 0.01,
    "learning_rate_multiplier": null
  },
  "organization_id": "org-5DvmRlg7kGvkcggchTvEJDF0",
  "model": "davinci",
  "training_files": [
    {
      "object": "file",
      "id": "file-kRRpt3KvorsOei3GZyFPDWQy",
      "purpose": "fine-tune",
      "filename": "training_data.jsonl",
      "bytes": 1310,
      "created_at": 1688016796,
      "status": "processed",
      "status_details": null
    }
  ],
  "validation_files": [],
  "result_files": [],
  "created_at": 1688016796,
  "updated_at": 1688018323,
  "status": "cancelled",
  "fine_tuned_model": null,
  "events": [
    {
      "object": "fine-tune-event",
      "level": "info",
      "message": "Created fine-tune: ft-nBpBzRcC3QZvXLftT2gUSISs",
      "created_at": 1688016796
    },
    {
      "object": "fine-tune-event",
      "level": "info",
      "message": "Fine-tune c

### Use your fine-tune model

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
fine_tuned_model = ""

new_prompt = "What is photosynthesis?"
response = get_completion(new_prompt)
print(response)

new_prompt = "Who is the prime minister of Thailand?"
response = get_completion(new_prompt)
print(response)
